In [1]:
import requests
from contextlib import closing
import csv
import json_stream.requests

In [2]:
url =  'https://zenodo.org/record/5887261'

# Transform URL for API request
record_id = url.split('/')[-1]
url = 'https://zenodo.org/api/records/' + record_id


In [3]:
try:
    r = requests.get(url, timeout=2) #timeout in seconds
except requests.exceptions.ConnectTimeout:
    print('Connection error')

if not r.ok:
    print('DOI could not be resolved. Try again, or use a different URL.')


In [4]:
if r.ok:
    js = r.json()
    files = js['files']
    total_size = sum(f['size'] for f in files)

In [5]:
total_size

48775784

In [6]:
for f in files:
    fname = f['key']
    link = 'https://zenodo.org/record/{}/files/{}'.format(
        record_id, fname
    ).replace(' ', '%20')
    print(fname, link)

Italian Municipality Codes.json https://zenodo.org/record/5887261/files/Italian%20Municipality%20Codes.json
NUTS_RG_01M_2021_4326_LEVL_0.geojson https://zenodo.org/record/5887261/files/NUTS_RG_01M_2021_4326_LEVL_0.geojson
NUTS_RG_01M_2021_4326_LEVL_1.geojson https://zenodo.org/record/5887261/files/NUTS_RG_01M_2021_4326_LEVL_1.geojson
NUTS_RG_01M_2021_4326_LEVL_2.geojson https://zenodo.org/record/5887261/files/NUTS_RG_01M_2021_4326_LEVL_2.geojson
NUTS_RG_01M_2021_4326_LEVL_3.geojson https://zenodo.org/record/5887261/files/NUTS_RG_01M_2021_4326_LEVL_3.geojson


In [7]:
url = 'https://zenodo.org/record/3694065/files/var10_30yrMeanAnnualCumulatedGDDAbove5degreesC_rcp26_2041_2070.csv'
url = 'https://zenodo.org/record/3694065/files/var13_AnnualVariationSolarRadiation_historical_1971_2005.csv?download=1'

In [12]:
url = 'https://zenodo.org/record/6958310/files/espeni.csv?download=1'

In [15]:

# For CSV files
with closing(requests.get(url, stream=True)) as r:
    f = (line.decode("utf-8") for line in r.iter_lines())
    reader = csv.reader(f, delimiter=",", quotechar='"')
    for idx, row in enumerate(reader):
       print(row)
       if idx>2:
        break


['ELEXM_SETTLEMENT_DATE', 'ELEXM_SETTLEMENT_PERIOD', 'ELEXM_utc', 'ELEXM_localtime', 'ELEXM_ROWFLAG', 'NGEM_ROWFLAG', 'POWER_ESPENI_MW', 'POWER_ELEXM_CCGT_MW', 'POWER_ELEXM_OIL_MW', 'POWER_ELEXM_COAL_MW', 'POWER_ELEXM_NUCLEAR_MW', 'POWER_ELEXM_WIND_MW', 'POWER_ELEXM_PS_MW', 'POWER_ELEXM_NPSHYD_MW', 'POWER_ELEXM_OCGT_MW', 'POWER_ELEXM_OTHER_POSTCALC_MW', 'POWER_ELEXM_BIOMASS_POSTCALC_MW', 'POWER_NGEM_EMBEDDED_SOLAR_GENERATION_MW', 'POWER_NGEM_EMBEDDED_WIND_GENERATION_MW', 'POWER_NGEM_BRITNED_FLOW_MW', 'POWER_NGEM_EAST_WEST_FLOW_MW', 'POWER_NGEM_MOYLE_FLOW_MW', 'POWER_NGEM_NEMO_FLOW_MW', 'POWER_NGEM_IFA_FLOW_MW', 'POWER_NGEM_IFA2_FLOW_MW', 'POWER_NGEM_NSL_FLOW_MW']
['2008-11-05', '43', '2008-11-05T21:00:00+00:00', '2008-11-05T21:00:00+00:00', '1', '1', '44020', '16468', '0', '19525', '5646', '62', '580', '314', '0', '0', '0', '0', '129', '0', '0', '-59', '', '1355', '', '']
['2008-11-05', '44', '2008-11-05T21:30:00+00:00', '2008-11-05T21:30:00+00:00', '1', '1', '42169', '15240', '0', '18

In [18]:
url = 'https://zenodo.org/record/5887261/files/Italian%20Municipality%20Codes.json'
#url = 'https://zenodo.org/record/5887261/files/NUTS_RG_01M_2021_4326_LEVL_0.geojson?download=1'

In [56]:
# For json/geosjon files
# see https://pypi.org/project/json-stream/

# Coding this basic function took me sooo long because I'm retarded :'(
def fill_sample_dic(data, max_depth = 5, depth = 0):
    if depth>=max_depth:
        return None
    # Case we're dealing with a node with several labeled fields
    dic = {}
    if type(data) == json_stream.base.PersistentStreamingJSONObject:
        for k, v in data.items():
            dic[k] = fill_sample_dic(v, max_depth = max_depth, depth = depth+1)
        return dic
    # Case we're dealing with a node with several unlabeled fields
    elif type(data) == json_stream.base.PersistentStreamingJSONList:
        for idx, v in enumerate(data):
            dic[idx] = fill_sample_dic(v, max_depth = max_depth, depth = depth+1)
        return dic
    else:
        return data



# Build a sample dictionnary of the data
with requests.get(url, stream=True) as response:
    data = json_stream.requests.load(response, persistent=True)
    dic = fill_sample_dic(data, max_depth = 2)

print(dic)



{'type': 'FeatureCollection', 'crs': {'type': None, 'properties': None}, 'features': {0: None, 1: None, 2: None, 3: None, 4: None, 5: None, 6: None, 7: None, 8: None, 9: None, 10: None, 11: None, 12: None, 13: None, 14: None, 15: None, 16: None, 17: None, 18: None, 19: None, 20: None, 21: None, 22: None, 23: None, 24: None, 25: None, 26: None, 27: None, 28: None, 29: None, 30: None, 31: None, 32: None, 33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 48: None, 49: None, 50: None, 51: None, 52: None, 53: None, 54: None, 55: None, 56: None, 57: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 65: None, 66: None, 67: None, 68: None, 69: None, 70: None, 71: None, 72: None, 73: None, 74: None, 75: None, 76: None, 77: None, 78: None, 79: None, 80: None, 81: None, 82: None, 83: None, 84: None, 85: None, 86: None, 87: None, 88: None, 89: None, 90: None, 91: None, 92: 